# Lyrics summarization pipeline

This notebook can be used to obtain a short description of a song meaniong based on its lyrics.

In [1]:
import pandas as pd
import numpy as np
import streamlit as st
import time
from langchain_groq import ChatGroq
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.chains import ConversationChain 

In [2]:
lyrics_df = pd.read_csv("data/cleaned_data/rag_dataset.csv")
lyrics_df['lyrics'] = lyrics_df['lyrics'].apply(lambda x: x.replace('\r', ''))
lyrics_df.head()

,song_name,album,happy_sad,relationship,feelings_of_self,glass_half_full,stages,tempo,seriousness,future_prospects,feelings_of_male,togetherness,lyrics,spotify_uri,url
0,cold as you,Taylor Swift,-10,-8,-1,-3,-3,-3,-3,-3,-1,-1,You have a way of coming easily to me\n And wh...,7an1exwMnfYRcdVQm0yDev,https://open.spotify.com/track/7an1exwMnfYRcdV...
1,i'm only me when i'm with you,Taylor Swift,9,10,3,3,1,2,2,2,3,3,Friday night beneath the stars\n In a field be...,0JIdBrXGSJXS72zjF9ss9u,https://open.spotify.com/track/0JIdBrXGSJXS72z...
2,invisible,Taylor Swift,-1,-4,0,-2,1,0,0,0,-1,-3,She can't see the way your eyes\n Light up whe...,5OOd01o2YS1QFwdpVLds3r,https://open.spotify.com/track/5OOd01o2YS1QFwd...
3,mary's song,Taylor Swift,5,12,0,2,1,2,3,3,3,3,She said I was seven and you were nine\n I loo...,2QrQCMel6v2JiLxqrg4p2O,https://open.spotify.com/track/2QrQCMel6v2JiLx...
4,our song,Taylor Swift,5,6,2,2,1,0,1,1,3,1,I was ridin' shotgun with my hair undone\n In ...,15DeqWWQB4dcEWzJg15VrN,https://open.spotify.com/track/15DeqWWQB4dcEWz...


In [26]:
groq_client = ChatGroq(temperature=0, model_name='mixtral-8x7b-32768', api_key=st.secrets["GROQ_API_KEY"])
conversation = ConversationChain(
        llm=groq_client,
)

In [27]:
prompt_template = ChatPromptTemplate.from_messages(
                [
                    SystemMessage(
                        content = ("""You are an AI assistant that has to provide a little description (with a maximum of 150 characters) of the meaning of a Taylor Swfit song given its lyrics.

                                   This is your only task. You just have to output a short description of the meaning of the song. The description must start with an open curly bracket and has to end with a closed curly bracket.
                        """)
                    ),
                    HumanMessagePromptTemplate.from_template("{text}")
                ]
            )

In [5]:
lyrics_test = lyrics_df.iloc[:2]['lyrics'].values
lyrics_test

array(["You have a way of coming easily to me\n And when you take, you take the very best of me\n So I start a fight 'cause I need to feel something\n And you do what you want 'cause I'm not what you wanted\n \n Oh, what a shame, what a rainy ending\n Given to a perfect day\n Just walk away, ain't no use defending\n Words that you will never say\n And now that I'm sitting here thinking it through\n I've never been anywhere cold as you\n \n You put up walls and paint them all a shade of gray\n And I stood there loving you and wished them all away\n And you come away with a great little story\n Of a mess of a dreamer with the nerve to adore you\n \n Oh, what a shame, what a rainy ending\n Given to a perfect day\n So just walk away, ain't no use defending\n Words that you will never say\n And now that I'm sitting here thinking it through\n I've never been anywhere cold as you\n   \n You never did give a damn thing, honey\n But I cried, cried for you\n And I know you wouldn't have told nob

In [6]:
query = lyrics_test[0]
prompt = prompt_template.format_messages(text=query)
response = conversation.invoke(prompt)
response

{'input': [SystemMessage(content='You are an AI assistant that has to provide a little description (with a maximum of 150 characters) of the meaning of a Taylor Swfit song given its lyrics.\n\n                                   This is your only task. You just have to output a short description of the meaning of the song. The description must start with an open curly bracket and has to end with a closed curly bracket.\n                        '),
  HumanMessage(content="You have a way of coming easily to me\n And when you take, you take the very best of me\n So I start a fight 'cause I need to feel something\n And you do what you want 'cause I'm not what you wanted\n \n Oh, what a shame, what a rainy ending\n Given to a perfect day\n Just walk away, ain't no use defending\n Words that you will never say\n And now that I'm sitting here thinking it through\n I've never been anywhere cold as you\n \n You put up walls and paint them all a shade of gray\n And I stood there loving you and wi

In [8]:
lyrics_df['description'] = None

In [72]:
start = 179
end = 180
for entry, row in enumerate(lyrics_df[start:end].iterrows()):
    print(f"Processing entry {entry+start}")
    query = row[1]['lyrics']
    prompt = prompt_template.format_messages(text=query)
    print(prompt[0])
    try:
        message = conversation.invoke(prompt)
        print(message.keys())
        lyrics_df.loc[row[0], 'description'] = message['response'].replace("Description: ", "").replace("{", "").replace("}", "").replace('"', "")
        time.sleep(20)
    except Exception as e:
        print(f'Error: {e}')
        entry-=1
        groq_client = ChatGroq(temperature=0, model_name='mixtral-8x7b-32768', api_key=st.secrets["GROQ_API_KEY"])
        conversation = ConversationChain(
                llm=groq_client,
        )
        time.sleep(30)
        

Processing entry 179
content='You are an AI assistant that has to provide a little description (with a maximum of 150 characters) of the meaning of a Taylor Swfit song given its lyrics.\n\n                                   This is your only task. You just have to output a short description of the meaning of the song. The description must start with an open curly bracket and has to end with a closed curly bracket.\n                        '
dict_keys(['input', 'history', 'response'])


In [73]:
pd.set_option('display.max_colwidth', None)
lyrics_df.iloc[120:][['song_name', 'description']]

,song_name,description
120,epiphany,"This song, possibly titled Epiphany, delves into the harrowing experiences of war and medical emergencies. It explores themes of sacrifice, resilience, and the human struggle to find meaning and understanding in the face of adversity."
121,exile,"This song, potentially titled The Film, seems to describe the painful experience of watching a relationship end, with the speaker recognizing the signs they've seen in a similar situation before. It touches on themes of heartbreak, unrequited love, and the struggle to move on."
122,hoax,"This song, possibly named My Only One, portrays a deep sense of longing and heartache. It describes the pain of a broken relationship, the memories that linger, and the struggle to move on from the past."
123,illicit affairs,"This song, potentially titled Illicit Affairs, explores the intense passion and ultimate heartache of a secret love affair. It delves into the deception, longing, and the consequences of such a relationship."
124,invisible string,"This song, possibly named Invisible String, reflects on the journey of life, love, and relationships. It uses the metaphor of an invisible string to depict the connection between two people, suggesting a destined bond despite time, distance, and past experiences."
125,mad woman,"This song, potentially titled Mad Woman, seems to express anger and defiance towards someone who provoked the speaker, leading her to feel intense emotions. It explores themes of resentment, retaliation, and the societal stigma around angry women."
126,mirrorball,"This song, possibly titled Mirrorball, uses the metaphor of a mirrorball to depict the speaker's adaptability and desire to please. It explores themes of self-presentation, resilience, and the longing for validation and connection."
127,my tears ricochet,"This song, potentially titled Tears Ricochet, seems to explore themes of heartache, betrayal, and the aftermath of a failed relationship. It delves into the speaker's feelings of resentment, loss, and the struggle to move on from the past."
128,peace,"This song, possibly titled Peace, delves into themes of love, loyalty, and the complexities of relationships. It explores the struggle to provide peace and happiness for a loved one, despite personal shortcomings and external challenges."
129,the 1,"This song, potentially titled The 1, reflects on a past relationship, exploring themes of nostalgia, longing, and what-ifs. It delves into the speaker's curiosity about how things could have been different if certain circumstances had changed."


In [74]:
# save csv as enchanced rag_dataset
lyrics_df.to_csv("data/cleaned_data/rag_dataset_enhanced.csv", index=False)

In [79]:
pd.set_option('display.max_colwidth', 10)
lyrics_df.head()

,song_name,album,happy_sad,relationship,feelings_of_self,glass_half_full,stages,tempo,seriousness,future_prospects,feelings_of_male,togetherness,lyrics,spotify_uri,url,description
0,cold a...,Taylor...,-10,-8,-1,-3,-3,-3,-3,-3,-1,-1,You ha...,7an1ex...,https:...,This s...
1,i'm on...,Taylor...,9,10,3,3,1,2,2,2,3,3,Friday...,0JIdBr...,https:...,This s...
2,invisible,Taylor...,-1,-4,0,-2,1,0,0,0,-1,-3,She ca...,5OOd01...,https:...,This s...
3,mary's...,Taylor...,5,12,0,2,1,2,3,3,3,3,She sa...,2QrQCM...,https:...,This s...
4,our song,Taylor...,5,6,2,2,1,0,1,1,3,1,I was ...,15DeqW...,https:...,This s...
